In [1]:
!pip install --upgrade vocos encodec librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.0 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45760 sha256=181af50b5f77b98f6dcc402947321b89a844b094b32474c6f70aafa1ce34dae9
  Stored in directory: /Users/nbilla/Library/Caches/pip/wheels/b4/a4/88/480018a664e58ca7ce6708759193ee51b017b3b72aa3df8a85
Successfully built encodec


In [1]:
import torch
from inference import Mars5TTS, InferenceConfig
import IPython.display as ipd
import torch
import librosa
import time

In [2]:
ar_url = "https://github.com/Camb-ai/mars5-tts/releases/download/v0.2/mars5_en_checkpoints_ar-1700000.pt"
nar_url = "https://github.com/Camb-ai/mars5-tts/releases/download/v0.2/mars5_en_checkpoints_nar-1750000.pt"

# download these models to './pretrained'

In [3]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    
print(f'Using device {device}')

Using device mps


In [4]:
def load_mars5(device=None, ar_path=None, nar_path=None) -> Mars5TTS:

    ar_ckpt = torch.load(str(ar_path), map_location='cpu')
    nar_ckpt = torch.load(str(nar_path), map_location='cpu')

    mars5 = Mars5TTS(ar_ckpt, nar_ckpt, device=device)
    return mars5, InferenceConfig

In [5]:
mars5, config = load_mars5(device=device, ar_path='./pretrained/mars5_en_checkpoints_ar-1700000.pt', nar_path='pretrained/mars5_en_checkpoints_nar-1750000.pt')

using device: mps


/Users/nbilla/miniconda3/envs/assisted_search/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/Users/nbilla/miniconda3/envs/assisted_search/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
# https://github.com/Camb-ai/mars5-tts/raw/master/docs/assets/example_ref.wav

In [6]:
wav, sr = librosa.load('./refernce_audio/example.wav', 
                       sr=mars5.sr, mono=True)
wav = torch.from_numpy(wav)
ref_transcript = "We actually haven't managed to meet demand."
print("Reference audio:")
ipd.display(ipd.Audio(wav.numpy(), rate=mars5.sr))
print(f"Reference transcript: {ref_transcript}")

Reference audio:


Reference transcript: We actually haven't managed to meet demand.


In [7]:
start = time.time()

deep_clone = True # set to False if you don't know prompt transcript or want fast inference.
# Below you can tune other inference settings, like top_k, temperature, top_p, etc...
cfg = config(deep_clone=deep_clone, rep_penalty_window=100,
                      top_k=100, temperature=0.7, freq_penalty=3)

ar_codes, wav_out = mars5.tts("The quick brown rat.", wav, 
          ref_transcript,
          cfg=cfg)

end = time.time()

print('Synthesized output audio:')
ipd.Audio(wav_out.numpy(), rate=mars5.sr)
print(f"time taken on : {device} :: {end - start}s")

Note: using deep clone. Assuming input `c_phones` is concatenated prompt and output phones. Also assuming no padded indices in `c_codes`.
New x: torch.Size([1, 1263, 8]) | new x_known: torch.Size([1, 1263, 8]) . Base prompt: torch.Size([1, 215, 8]). New padding mask: torch.Size([1, 1263]) | m shape: torch.Size([1, 1263, 8])


/Users/nbilla/miniconda3/envs/assisted_search/lib/python3.11/site-packages/torch/nn/functional.py:4838: UserWarning: The operator 'aten::col2im' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn.col2im(


Synthesized output audio:
time taken on : mps :: 452.77952694892883s


# CPU
time taken on : cpu :: 573.9996061325073s
# MPS
time taken on : mps :: 452.77952694892883s
# GPU (Colab - T4)